# 16-1. 들어가며

### Open Set Recognition과 쏘카

___

쏘카의 정위치 주차 판별 문제를 해결하기 위한 방법을 탐색하던 중 Open Set Recognition을 만났습니다.

Open Set Recognition은 연구 범위가 넓고, 연구마다 약간씩 다른 상황과 목적을 가지고 있습니다. 여러 가지 연구 중 우리는 OpenMax라는 기법을 구현해 볼 예정입니다. OpenMax는 Open Set Recognition이 논의되던 초기 연구라 성능이 우수하지는 않지만, 현실 문제를 해결하고자 하는 모습이 매우 흥미로울 겁니다.

실험실의 Closed Set 환경을 벗어나 현실과 가까운 Open Set 문제를 맞이하고 나면 좀 더 현실 문제를 위한 인공지능을 들여다볼 수 있을 겁니다. 열심히 인공지능을 공부해오며 과연 현실 세계에서 어떤 모습으로 활용될 수 있을지 궁금했을 텐데요. 여러분은 이제 그 넓은 세계로 나아가는 발판에 서 있는 것입니다. 그럼 시작해 봅시다.

### 학습 목표

___

-   Open Set Recognition의 기본 개념을 이해한다.
-   Open Set Recognition에 이용되는 기술들을 알아본다.
-   OpenMax 기법을 적용해본다.

### 학습 목차

___

1.  OpenMax
2.  1단계: Activation Vector 추출
3.  통계적 방법
4.  2단계: Weibull-Distribution
5.  OpenMax 적용
6.  더 넓은 세상
7.  프로젝트

### 준비물

___

폴더를 생성하고 준비된 데이터를 연결해 주세요. 이미 준비하신 분은 넘어가도 좋습니다.

```bash
$ mkdir -p ~/aiffel/socar_open_set/data
$ mkdir -p ~/aiffel/socar_open_set/weights
$ ln -s ~/data/data/* ~/aiffel/socar_open_set/data
$ ln -s ~/data/weights/* ~/aiffel/socar_open_set/weights
```

이전다음

✋

# 16-2. OpenMax

### OpenMax의 등장

___

앞서 알아본 Open Set Recognition 문제를 해결하는 방법은 매우 많습니다. Open Set Recognition 문제를 해결하기 위해 딥러닝 모델을 사용한 방법 중 초기에 등장한 OpenMax라는 기법을 구현해 볼 예정입니다. OpenMax는 딥러닝 모델과 통계적 방법을 함께 사용한 방법입니다. 아래의 **Towards Open Set Deep Networks**라는 논문으로 발표되었으니 참고하세요!

-   [논문: Towards Open Set Deep Networks](https://arxiv.org/pdf/1511.06233.pdf)
-   [영상: Towards Open Set Deep Networks](https://www.youtube.com/watch?v=41YrOiavhGc)  
    (새...샘숭?! 그렇습니다. 이미 오래전부터 그곳에서는...)

### OpenMax의 구성

___

OpenMax를 두 파트로 나누면 딥러닝 모델에서 **Activation Vector를 추출**하는 단계와 **Activation Vector를 통계적 기법에 사용**하는 단계로 나뉩니다.

-   [논문 리뷰 1: Towards open set deep networks](https://october25kim.github.io/paper/openset/2020/10/11/openmax-paper/)
-   [논문 리뷰 2: Towards open set deep networks](https://bigdata-analyst.tistory.com/m/258?category=908123)

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-6-p-1-1.max-800x600.png)

\[Softmax에 들어가는 것이 Activation Vector\]

[https://towardsdatascience.com/deep-learning-concepts-part-1-ea0b14b234c8](https://towardsdatascience.com/deep-learning-concepts-part-1-ea0b14b234c8)

우선 첫 단계에서는 딥러닝 모델을 통해 데이터의 Activation Vector를 추출합니다. 여기서 Activation Vector는 Softmax값이 되기 전 Vector를 의미해요. 이미지를 딥러닝 모델에 입력하고 Softmax 층 직전 값을 모아두면 됩니다.

다음 단계에서는 통계적 방법을 사용합니다. OpenMax에서 사용하는 통계적 방법은 Extreme Value Theory와 Weibull-Distribution이 주요 내용인데요. 지금은 우선 이름만 확인하고 넘어갈게요. 뒤에서 더 자세히 다루도록 하겠습니다. 통계적 방법들은 어렵지만, 너무 깊게 이해하지 않아도 됩니다. 현재는 Open Set Recognition에서 OpenMax가 차지하는 비중이 크지 않으니 개괄적으로만 이해해도 충분합니다.

그런데 딥러닝 모델을 통해서 Activation Vector를 추출한 후 어떤 방식으로 이용해야 '분류하지 않을' 수 있을까요? '분류하지 않기' 위해서 `reject`라는 클래스를 하나 더 만듭니다. 딥러닝 모델에 n개의 클래스가 있을 때, 통계적 방법을 통과하고 나면 n+1개의 클래스가 나오게 됩니다. 딥러닝 모델은 학습 데이터를 모두 분류해서 Activation Vector를 얻어내고, 통계적 방법을 통해 '분류하지 않음'이 가능하도록 할 겁니다.

이전다음

✋

# 16-3. 1단계: Activation Vector 추출

### 모델에서 Activation Vector를 추출하자

___

OpenMax를 두 부분으로 나누었을 때, 앞 부분에서는 딥러닝 모델을 통해 데이터의 Activation Vector를 추출한다고 했습니다. 이제 딥러닝 모델을 써서 Activation Vector를 추출해 봅시다.

먼저 필요한 라이브러리를 불러옵니다.

In [1]:
import os
import numpy as np
from statistics import mean

import torch
import torchvision

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from scipy import stats

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

PROJECT_PATH = os.getenv('HOME') + '/aiffel/socar_open_set'
MODEL_PATH = os.path.join(PROJECT_PATH, 'weights')
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
REJECT_PATH = os.path.join(DATA_PATH, 'reject')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # 여기서 'cuda'가 출력되어야 GPU와 연결이 됩니다

cuda


데이터 전처리 파이프라인에는 이전에 사용했던 `create_dataloader`를 그대로 사용합시다.

In [ ]:
def create_dataloader(path, batch_size, istrain):
    nearest_mode = torchvision.transforms.InterpolationMode.NEAREST
    normalize = torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
    train_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((320,320), interpolation=nearest_mode),
        torchvision.transforms.CenterCrop((224,224)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ColorJitter(),
        torchvision.transforms.ToTensor(),
        normalize
    ])

    test_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((320,320), interpolation=nearest_mode),
        torchvision.transforms.CenterCrop((224,224)),
        torchvision.transforms.ToTensor(),
        normalize
    ])
    
    if istrain:
        data = torchvision.datasets.ImageFolder(path, transform=train_transformer)
        dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
        
    else:
        data = torchvision.datasets.ImageFolder(path, transform=test_transformer)
        dataloader = torch.utils.data.DataLoader(data, shuffle=False)

    return dataloader, data

print('슝=3')

학습 데이터를 연결하고, 미리 준비해 둔 모델을 불러옵니다. 이 모델은 이전에 사용했던 이미지 분류 모델입니다. 클래스가 `atower_b5`, `balsan_b5`, `balsan_b6`, `dcube_b6`로 총 4개인 모델이에요.

In [ ]:
train_loader, _train_data = create_dataloader(TRAIN_PATH, 1, False)
target_class_num = len(os.listdir(TRAIN_PATH))

net = torchvision.models.resnet50(pretrained=True)
net.fc = torch.nn.Linear(
    net.fc.in_features,
    target_class_num
)

saved_weight_path = os.path.join(MODEL_PATH, 'classifier_acc_0.96008.pth')
net.load_state_dict(torch.load(saved_weight_path, map_location=device))
print('Successfully Loaded the Network Weight!')
net.eval()

net.to(device)

이제 데이터와 모델이 준비되었으니 OpenMax에 사용할 데이터를 추출할 수 있습니다.

OpenMax에 필요한 데이터는 **분류에 성공한 데이터의 Activation Vector**라는 점을 꼭 기억해야 합니다. Softmax 층에 입력되는 값이 Activation Vector이니 `torch.softmax()`의 입력이 되는 값에서 뽑아오면 되겠습니다.

시간이 꽤 걸립니다.

In [ ]:
train_preds = list()
train_actvecs = list()
train_outputs_softmax = list()
train_labels = list()

with torch.no_grad():
  for idx, (img, label) in enumerate(train_loader):
      img = img.to(device)
      label = label.to(device)

      out = net(img)
      out_actvec = out.cpu().detach().numpy()[0]
      out_softmax = torch.softmax(out, 1).cpu().detach().numpy()[0]
      out_pred = int(torch.argmax(out).cpu().detach().numpy())
      out_label = int(label.cpu().detach().numpy())

      train_actvecs.append(out_actvec) # component 1: softmax 전의 Activation Vector
      train_preds.append(out_pred) # componenet 2: 각 데이터에 대한 예측값
      train_outputs_softmax.append(out_softmax) # component 3: 각 데이터에 대한 softmax 확률
      train_labels.append(out_label) # component 4: 각 데이터에 대한 Label (정답)

train_actvecs = np.asarray(train_actvecs)
train_preds = np.asarray(train_preds)
train_outputs_softmax = np.asarray(train_outputs_softmax)
train_labels = np.asarray(train_labels)

print('슝=3')

중요한 것이 하나 있습니다. 모델에서 나온 Activation Vector를 모두 사용하지 않는다는 점입니다. 통계적 방법에 따라 다를 수 있는데요. OpenMax에서는 모델이 **정답을 맞힌 Activation Vector**만 사용합니다. 올바른 경우의 Activation Vector만 사용하겠다는 이야기죠.

In [ ]:
train_correct_actvecs = train_actvecs[train_labels==train_preds]
train_correct_labels = train_labels[train_labels==train_preds]
print('Activation vector: ', train_correct_actvecs.shape)
print('Labels: ', train_correct_labels.shape)

이제 OpenMax를 위한 데이터가 준비되었습니다. 이 데이터를 어떻게 통계적 방법에 적용하는지 살펴봅시다.


# 16-4. 통계적 방법

### 평범하지 않다면 천재 아니면 바보다?!

___

이미지를 분류 모델에 넣었을 때 reject하기 위해서 판단하는 기준은 "평범한가?"입니다. 평범하지 않다면 무언가 잘못되었다고 보는 거죠. 그게 좋든 나쁘든 말이에요. 그래서 평범의 기준을 만들어야 합니다. 어떤 수리 모델이든 상관없습니다. 아주 쉽게는 정규분포를 떠올릴 수도 있어요. 어떤 사람의 성격, 태도, 행동, 지능 등등을 모두 수치화 했을 때 정규분포에서 아주 끝에 있다면 그 사람은 천재 아니면 바보 아닐까요??

그렇다면 각각의 값이 분포의 끝에 있는지를 어떻게 수학적으로 측정하고 엄밀하게 판단할 수 있을까요? 이때 사용되는 것이 **Extreme Value Theory**입니다. Extreme Value Theorem과 다르니 주의하세요!!

-   [Extreme Value Theory](https://en.wikipedia.org/wiki/Extreme_value_theory)

Extreme Value Theory(EVT)는 Extreme Value Analysis(EVA)라고도 불리는데요. 핵심적인 내용을 간추리면 평균으로부터 거리를 계산하면 이 값이 극단값일 확률을 알 수 있다는 것입니다. 즉 천재이거나 바보일 확률을 결정할 수 있는 수학적이고 통계적인 방법이라고 생각하면 됩니다. 중요한 점은 **평균으로부터의 거리를 이용**한다는 점입니다. 그렇다면 평균으로부터의 거리가 식에 이용이 되겠네요!

Extreme Value Theory는 원래 위험 예측을 위해 자주 사용되고, 기계 고장, 금융 사고, 자연재해 등의 연구에 많이 인용됩니다. 측정이 자주 되지 않는 극단값 연구들이라는 공통점이 있네요. 우리가 원하는 `reject`도 그러한 성질이 있으니 극단값의 개념을 차용한 것이라고 볼 수 있습니다.

Extreme Value Theory에 주로 사용되는 분포는 Frechet distribution, Weibull distribution, Gumbel distribution, 이 세 가지가 있는데, OpenMax는 그 중 베이불 분포(Weibull distribution)를 사용했습니다. 정확히는 OpenMax가 인용한 Meta-Recognition에서 베이불 분포를 사용했지요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/original_images/so-6-p-3-2.png)

\[OpenMax는 Meta-Recognition을 이용하고 Meta-Recognition은 베이불 분포를 이용합니다\]

[https://arxiv.org/pdf/1511.06233.pdf](https://arxiv.org/pdf/1511.06233.pdf)

-   [Meta-Recognition: The Theory and Practice of Recognition Score Analysis](https://apps.dtic.mil/sti/pdfs/ADA534235.pdf)
-   [What is meta-recognition and Extreme Value Theory?](https://medium.com/@cdsjatin/what-is-meta-recognition-and-what-is-extreme-value-theory-1ebf296ee13a)

Meta-Recognition에서는 아래 그림처럼 Extreme Value Theory를 사용했습니다. 아래 그림에서 Tail Analysis에 베이불 분포를 사용했다고 생각하면 되겠네요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/original_images/so-6-p-3-3.png)[https://apps.dtic.mil/sti/pdfs/ADA534235.pdf](https://apps.dtic.mil/sti/pdfs/ADA534235.pdf)

베이불 분포는 원래 제품의 고장 확률을 다루기 위해 고안되었습니다. 너무 자세히 알 필요는 없고, 정규 분포처럼 확률을 나타내는 어떤 분포인데 특히 제품 고장 상황을 예측하기 위한 분포라고 여기고 넘어가도 좋아요.

-   [Weibull distribution](https://en.wikipedia.org/wiki/Weibull_distribution)

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-6-P-3-1.max-800x600.png)

\[베이불 분포의 특징\]

[https://link.springer.com/article/10.1007/s10928-020-09721-0](https://link.springer.com/article/10.1007/s10928-020-09721-0)

위에서도 언급했듯이 Extreme Value Theory에는 세가지 분포가 사용되지만, 어떤 상황이냐에 따라 서로 다른 분포를 사용할 수 있습니다. 그중 베이불 분포가 한 쪽으로만 꼬리를 갖고 다른 쪽으로는 값이 0이 되는 특징이 있기 때문에 Meta-Recognition에서 이용되었고, OpenMax에서도 그대로 이용됩니다.

여기까지 이론이 이해가 되지 않았더라도 넘어가도 좋습니다. `reject`를 하기 위해서 얼마나 분포의 끝에 있는지 수치로 표현할 수 있는 베이불 분포를 사용한다는 점만 알면 됩니다. 베이불 분포는 이미 라이브러리 형태로 제공되고 있으니까요!

이전다음

✋

# 16-5. 2단계: Weibull-Distribution

### 적용해 봅시다

___

배경 이론을 이해했다면 적용해 봅시다.

앞서 추출해 낸 Activation Vector를 기억하고 있겠죠? 이 Vector들은 아래처럼 이용됩니다.

1.  Activation Vector를 클래스마다 나눠 담습니다.
2.  클래스별로 나눠진 Activation Vector별 평균으로부터 가장 먼 100개의 Vector를 이용해 베이불 분포의 모수를 추출합니다.
3.  각 클래스당 베이불 분포의 모수들을 저장해 둡니다.

우리가 사용할 베이불 분포의 모수는 `shape`, `loc`, `scale`로 3개이고, 클래스는 4개이므로 총 12개의 숫자가 있을겁니다. 아직 클래스가 5개가 되지 않았다는 점에 유의하세요!

아래 코드를 통해 베이불 분포의 모수를 얻을 수 있습니다.

In [ ]:
class_means = list()
dist_to_means = list()
mr_models = {}

for class_idx in np.unique(train_labels):
    
    print('class_idx: ', class_idx)
    class_act_vec = train_correct_actvecs[train_correct_labels==class_idx]
    print(class_act_vec.shape)
    
    class_mean = class_act_vec.mean(axis=0)
    class_means.append(class_mean)
    
    dist_to_mean = np.square(class_act_vec - class_mean).sum(axis=1) # 각 activation vector의 거리를 계산
    dist_to_mean_sorted = np.sort(dist_to_mean).astype(np.float64) # 거리를 기준으로 오름차순 정렬
    dist_to_means.append(dist_to_mean_sorted)

    shape, loc, scale = stats.weibull_max.fit(dist_to_mean[-100:]) # 거리가 가장 먼 100개를 사용하여 모수 추출
    
    mr_models[str(class_idx)] = {
        'shape':shape,
        'loc':loc,
        'scale':scale
    }
    
class_means = np.asarray(class_means)

이렇게 얻은 모수는 `shape`, `loc`, `scale`인데요. 각각 역할이 다릅니다.

`shape`은 베이불 분포의 모양을 결정하는데요. 베이불 분포에서는 아래처럼 굉장히 다른 모양들이 나타날 수 있습니다.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-6-p-4-1.max-800x600.jpg)

\[Shape에 따른 베이불 분포\]

[https://applicationsresearch.com/WeibullEase.htm](https://applicationsresearch.com/WeibullEase.htm)

`loc`은 분포의 가로축 평행 이동을 뜻하고, `scale`은 분포가 얼마나 넓게 퍼져있는지는 뜻합니다.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-6-p-4-2.max-800x600.png)

\[Scale이 클 수록 넓게 퍼집니다\]

[https://www.boost.org/doc/libs/1\_56\_0/libs/math/doc/html/math\_toolkit/dist\_ref/dists/weibull\_dist.html](https://www.boost.org/doc/libs/1_56_0/libs/math/doc/html/math_toolkit/dist_ref/dists/weibull_dist.html)

정상적으로 분류된 Activation Vector를 이용해 베이불 모수를 얻었다는 것은 극단값을 판별할 기준을 얻었다는 이야기입니다. 이제 남은 일은 이 기준을 통해 데이터를 분류하는 일뿐이네요!

이전다음

✋

# 16-6. OpenMax 적용

앞서 계산된 모수는 극단값을 판별할 기준이라고 할 수 있습니다. 지금까지 우리는 각 클래스마다 극단값의 기준을 설정했어요.

하나의 클래스에서 평범하다고 판별된다는 것은 그 클래스에 속한다는 이야기이죠. 반대로 모든 클래스에서 평범하지 않다고 판단된다면 어떤 클래스에도 속하지 않는다는 이야기이니 `reject`라는 클래스로 분류해 버리면 우리의 목적을 달성할 수 있겠네요!

OpenMax 기법의 핵심 부분이 완성되었습니다. 이제 OpenMax 확률을 계산하는 함수를 만들어 봅시다.

### OpenMax 확률

___

지금까지 계산한 것은 베이불 분포의 모수 뿐입니다. 이제 우리가 정위치/오위치를 판별하고자 하는 이미지와 베이불 분포의 모수를 이용해서 최종 분류를 할 수 있는 기준을 만들어야 합니다.

우선 베이불 분포로부터 이미지의 확률을 계산합니다. 베이불 분포 확률이 높다는 이야기는 평범하지 않다고 판별하는 것이니까요. 확률이 낮을수록 평범하다는 이야기입니다.

weight×ActivationVectorweight \\times ActivationVectorweight×ActivationVector

1에서 각 이미지의 확률(score)을 뺀 값을 가중치(weight)라 하고, 이 가중치를 Activation Vector에 곱해줍니다. 만약 각 이미지가 정해진 클래스에 속할수록 이 값은 높은 값이 나옵니다. 반대로 어떤 클래스에 속하지 않을수록 낮은 값이 나오죠. 마지막에 reject class를 추가하여 새로운 값을 추가해 줍니다.

∑(1−weight)(ActivationVector)\\sum {(1-weight)(ActivationVector)}∑(1−weight)(ActivationVector)

위 식을 계산하면 reject class에 속할수록 높은 값이 나옵니다. 만약 모든 클래스에 포함될 확률이 낮다면 이 값만 높게 나올 거예요. 분류하지 않을 수록 높은 값이 도출된 것이죠. 이걸 `reject`확률로 결정합시다.

이 과정에서 드디어 훈련 클래스보다 하나 더 많은 클래스 개수가 생성됩니다. `reject` 클래스가 추가된 것이죠.

이해가 되었다면 코드를 작성해 봅시다.

In [ ]:
def compute_openmax(actvec, class_means, mr_models):
    dist_to_mean = np.square(actvec - class_means).sum(axis=1)

    scores = list()
    for class_idx in range(len(class_means)):
        params = mr_models[str(class_idx)]
        score = stats.weibull_max.cdf(
            dist_to_mean[class_idx],
            params['shape'],
            params['loc'],
            params['scale']
        )
        scores.append(score)
    scores = np.asarray(scores)
    
    weight_on_actvec = 1 - scores # 각 class별 가중치
    rev_actvec = np.concatenate([
        weight_on_actvec * actvec, # known class에 대한 가중치 곱
        [((1-weight_on_actvec) * actvec).sum()] # unknown class에 새로운 계산식
    ])
    
    openmax_prob = np.exp(rev_actvec) / np.exp(rev_actvec).sum()
    return openmax_prob

print('슝=3')

### Inference 함수

___

학습한 클래스가 n 개 일 때, n+1개의 클래스의 확률을 알려주는 함수까지 만들어 졌습니다. 확률이 최대가 되는 경우의 클래스만 반환하면 될 것 같지만, 한 가지 더 보완해 주도록 하겠습니다. 바로 threshold값입니다.

계산한 최대 확률이 모두 다 낮은 경우라면 강제로 `reject`클래스로 분류해주는 방법입니다. 그 기준이 되는 값이 threshold입니다. OpenMax가 등장한 것이 2015년인데, 이 당시 threshold은 자주 사용되었습니다. 물론 단점이 많았습니다. 이 threshold값을 하나하나 찾아봐야 했기 때문입니다. 하지만 이번에는 그 때로 돌아가 threshold를 찾는 어려움이 있더라도 일단 함수를 작성해 봅시다.

In [ ]:
def inference(actvec, threshold, target_class_num, class_means, mr_models):
    openmax_prob = compute_openmax(actvec, class_means, mr_models)
    openmax_softmax = np.exp(openmax_prob)/sum(np.exp(openmax_prob))

    pred = np.argmax(openmax_softmax)
    if np.max(openmax_softmax) < threshold:
        pred = target_class_num
    return pred    

print('슝=3')

### Threshold 탐색을 위해

___

Threshold 탐색을 쉽게 하기 위해 함수 하나만 더 만들어 봅시다.

In [ ]:
def inference_dataloader(net, data_loader, threshold, target_class_num, class_means, mr_models, is_reject=False):
    result_preds = list()
    result_labels = list()

    with torch.no_grad():
      for idx, (img, label) in enumerate(data_loader):
          img = img.to(device)
          label = label.to(device)

          out = net(img)
          out_actvec = out.cpu().detach().numpy()[0]
          out_softmax = torch.softmax(out, 1).cpu().detach().numpy()[0]
          out_label = int(label.cpu().detach().numpy())

          pred = inference(out_actvec, threshold, target_class_num, class_means, mr_models)
      
          result_preds.append(pred)
          if is_reject:
              result_labels.append(target_class_num)
          else:
              result_labels.append(out_label)

    return result_preds, result_labels

print('슝=3')

이제 모든 준비가 끝났습니다. 이제 남은 것은 적당한 threshold를 찾아 성능을 찾아보는 일 뿐입니다.

그런데 탐색해야 할 성능의 기준도 중요합니다. Open Set Recognition에서의 성능은 `reject`를 구분하는 능력만을 뜻하지 않습니다. 일반 모델이 가진 분류 능력을 최대한 유지하면서 `reject`에 대한 판별도 높아야 합니다.

우리가 관심을 가지고 있는 데이터셋은 크게 두 종류가 있습니다.

-   정위치에 주차된 이미지
-   오위치에 주차된 이미지

이 경우 우리가 만든 모델의 성능을 평가하기 위해서는

-   정위치에 주차된 이미지들을 알맞은 class로 추론했는지
-   오위치에 주차된 이미지들을 추론하지 않고 reject 했는지

의 두 가지 조건을 살펴보아야 합니다. 이 두 가지 조건을 가장 잘 만족하는 threshold를 찾아야 합니다.

우선 0.3으로 threshold를 정하고 탐색해 봅시다. 조금 오래 걸립니다.

In [ ]:
test_loader, _test_data = create_dataloader(TEST_PATH, 1, False)
reject_loader, _reject_data = create_dataloader(REJECT_PATH, 1, False)
target_class_num = len(os.listdir(TEST_PATH))

test_preds, test_labels = inference_dataloader(net, test_loader, 0.3, target_class_num, class_means, mr_models)
reject_preds, reject_labels = inference_dataloader(net, reject_loader, 0.3, target_class_num, class_means, mr_models, is_reject=True)

print('Test Accuracy: ', accuracy_score(test_labels, test_preds))
print('Reject Accuracy: ', accuracy_score(reject_labels, reject_preds))

어떤가요? 다른 threshold와 비교할 수 없으니 threshold를 약간 올려서 살펴 볼까요? 0.4로 해봅시다.

In [ ]:
test_preds, test_labels = inference_dataloader(net, test_loader, 0.4, target_class_num, class_means, mr_models)
reject_preds, reject_labels = inference_dataloader(net, reject_loader, 0.4, target_class_num, class_means, mr_models, is_reject=True)

print('Test Accuracy: ', accuracy_score(test_labels, test_preds))
print('Reject Accuracy: ', accuracy_score(reject_labels, reject_preds))

오위치 이미지에 대한 성능은 크게 증가했지만, 정위치 이미지에 대한 성능이 너무 하락한 것 같네요. 중간 값인 0.35정도로 해볼까요?

In [ ]:
test_preds, test_labels = inference_dataloader(net, test_loader, 0.35, target_class_num, class_means, mr_models)
reject_preds, reject_labels = inference_dataloader(net, reject_loader, 0.35, target_class_num, class_means, mr_models, is_reject=True)

print('Test Accuracy: ', accuracy_score(test_labels, test_preds))
print('Reject Accuracy: ', accuracy_score(reject_labels, reject_preds))

어떤가요? 만족할만 한가요? 모델의 성능이 threshold에 따라 변하는걸 보니 적당한 threshold값을 찾는 것도 매우 중요해 보입니다!!


# 16-7. 더 넓은 세상

OpenMax의 결과가 만족스러운가요? OpenMax는 2015년 초기에 제안된 방법임을 고려해보면 그 동안 얼마나 많은 발전이 있었을지 가늠하기 어렵습니다.

OpenMax가 시작된 시점과 최근의 연구들을 종합해보면 꽤나 놀랍습니다. 아래 링크를 통해 확인해 봅시다.

-   [Awesome Open Set Recognition list](https://github.com/iCGY96/awesome_OpenSetRecognition_list)

최근의 연구는 딥러닝 모델의 loss에 변화를 주어 한 번의 학습만으로도 가능하도록 진행이 되고 있습니다. 한 번의 학습과 추론으로 Open Set Recognition을 해결하려고 하고 있죠.

-   [Class Anchor Clustering: A Loss for Distance-based Open Set Recognition](https://arxiv.org/pdf/2004.02434v3.pdf)
-   [Classification-Reconstruction Learning for Open-Set Recognition](https://arxiv.org/pdf/1812.04246.pdf)

![content img](https://d3s0tskafalll9.cloudfront.net/media/original_images/so-6-p-6-1.png)

\[이렇게 loss를 바꾸기도 합니다.\]

[https://arxiv.org/pdf/2006.15117.pdf](https://arxiv.org/pdf/2006.15117.pdf)

적대적 생성 모델을 활용한 방법도 활발하게 연구 되고 있습니다.

-   [Generative-discriminative Feature Representations for Open-set Recognition](https://openaccess.thecvf.com/content_CVPR_2020/papers/Perera_Generative-Discriminative_Feature_Representations_for_Open-Set_Recognition_CVPR_2020_paper.pdf)

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-6-p-6-2.max-800x600.png)

\[가짜를 만들어서 Unknown으로 학습시켜보자!\]

[https://arxiv.org/pdf/2103.00953.pdf](https://arxiv.org/pdf/2103.00953.pdf)

최근에는 분류 문제를 넘어 세그멘테이션 등의 다른 문제에까지도 적용되고 있습니다.

-   [Exemplar-Based Open-Set Panoptic Segmentation Network](https://arxiv.org/pdf/2105.08336v2.pdf)

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-6-p-6-3.max-800x600.png)

\[무엇을 Unknown으로 하느냐에 따라 달라져요.\]

[https://arxiv.org/pdf/2006.14673.pdf](https://arxiv.org/pdf/2006.14673.pdf)

위와 같이 다양한 연구를 통해 알 수 있는 사실은, 각 연구가 해결하고자 하는 문제들이 서로 다르다는 것입니다. 세상에는 해결해야 할 문제가 많은 만큼 해결 방법도 다양합니다. 비슷한 주제로 보이지만, 자세히 들여다보면 전혀 다른 관점과 목표를 가지고 있습니다. 이 때문에 Open Set Recognition 분야는 더 활발히 연구가 진행될 수밖에 없고, 더 많은 방법들이 제안될 것으로 보입니다. 굉장히 매력적인 분야가 아닐까요~!?

Closed Set에서 성과를 낸 딥러닝 모델이 실제 세계에 서서히 적용되는 것처럼, 아이펠에서 열심히 공부한 여러분도 넓은 세상에서 서서히 그 능력을 발휘하기를 기대하며 응원을 보냅니다~!

이전다음

✋

# 16-8. 프로젝트

지금까지 OpenMax를 적용해 보는 과정을 살펴 보았습니다.

OpenMax는 딥러닝 모델을 이용하는 단계와 통계적 방법을 이용하는 단계가 나누어져 있기 때문에, 두 단계가 모두 성능에 영향을 미칩니다.

이제 여러분의 모델로 OpenMax를 적용해 보는 시간을 가집시다.

먼저 주요 라이브러리를 살펴봅시다.

In [ ]:
import torch
import torchvision
import numpy as np
import PIL

print(torch.__version__)
print(torchvision.__version__)
print(np.__version__)
print(PIL.__version__)

### OpenMax 적용해보기

___

기본이 되는 딥러닝 모델을 학습하는 방법부터 여러분만의 방법을 사용해 봅시다. Augmentation부터 모델 훈련에 이르기까지 모든 것을 여러분 스스로 해봅시다.

모델을 학습하고 Activation Vector를 얻어냈다면 OpenMax를 적용해 봅니다. 중간 함수는 그대로 이용해도 좋습니다. 다만 각자 최적의 threshold값을 찾아야 할 겁니다!

### 좋은 성능을 찾아보자

___

Test Accuracy가 0.85 이상이면서 reject accuracy가 최대가 되는 threshold를 찾아봅시다. threshold를 미세하게 조정할수록 좋은 결과를 얻을 수 있어요.

### Threshold에 따른 추이를 살펴보자

___

Threshold가 변함에 따라 test accuracy와 reject accuracy가 변화합니다. 이런 변화를 잘 관찰하고 분석해 봅시다.